In [67]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt

In [68]:
# Parameters
IMG_SIZE = (200, 200)
BATCH_SIZE = 32
DATA_DIR = "D:\Soft\kagglecatsanddogs_3367a\PetImages"
EPOCHS = 20

In [69]:
# Helper function to handle corrupted images and sanitize paths
def is_valid_image(file_path):
    try:
        img = load_img(file_path)
        return True
    except Exception:
        return False

In [70]:
def sanitize_path(path):
    return path.encode("utf-8", "ignore").decode("utf-8")

In [71]:
def load_and_preprocess_data(data_dir):
    classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d)) and d.startswith('n') and d[1:].isdigit()]
    valid_images = []
    labels = []

    for label, class_name in enumerate(classes):
        class_dir = sanitize_path(os.path.join(data_dir, class_name))
        for file_name in os.listdir(class_dir):
            file_path = sanitize_path(os.path.join(class_dir, file_name))
            if is_valid_image(file_path):
                valid_images.append(file_path)
                labels.append(label)

    return valid_images, labels, classes

In [72]:
# Dataset preparation
image_paths, labels, class_names = load_and_preprocess_data(DATA_DIR)
dataset_size = len(image_paths)
indices = np.arange(dataset_size)
np.random.shuffle(indices)

split_index = int(0.8 * dataset_size)
train_indices = indices[:split_index]
test_indices = indices[split_index:]

train_paths = [image_paths[i] for i in train_indices]
train_labels = [labels[i] for i in train_indices]
test_paths = [image_paths[i] for i in test_indices]
test_labels = [labels[i] for i in test_indices]

train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True, rotation_range=15)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 24990 images belonging to 2 classes.
Found 24990 images belonging to 2 classes.


In [73]:
# Model creation
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [74]:
# Model training
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS
)

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'ArgMax_1' defined at (most recent call last):
    File "C:\Users\admin\miniconda3\envs\gpu\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\admin\miniconda3\envs\gpu\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\admin\miniconda3\envs\gpu\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\admin\miniconda3\envs\gpu\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\admin\miniconda3\envs\gpu\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\admin\miniconda3\envs\gpu\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\admin\miniconda3\envs\gpu\lib\asyncio\base_event

In [ ]:
# Plotting metrics
def plot_metrics(history):
    plt.figure(figsize=(12, 4))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss')

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy')

    plt.show()

plot_metrics(history)

In [ ]:
# Save the model
model.save("image_classifier_vgg16.h5")